In [1]:
import os
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from enum import Enum
import cv2
from tensorflow.keras import backend as K # Importing Keras backend (by default it is Tensorflow)
import gc
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD, Adam
from keras_adabound import AdaBound
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Input, Conv2D,BatchNormalization
from tensorflow.keras.layers import Activation,SpatialDropout2D,AvgPool2D
from tensorflow.keras.layers import MaxPool2D,Dropout,GlobalAveragePooling2D
from tensorflow.keras.layers import GlobalMaxPooling2D,Flatten,Dropout,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Sequential
from sklearn.metrics import cohen_kappa_score
from tensorflow.keras.metrics import AUC, BinaryAccuracy
from sklearn.metrics import cohen_kappa_score, accuracy_score
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.applications import DenseNet121,DenseNet169,DenseNet201
from tensorflow.keras.models import load_model
from numpy import dstack
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50

In [2]:
seed = 3352024
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)


In [3]:
"""Load Data"""

import os
import pandas as pd

def load_path(path):
    '''
    Load bone fracture dataset
    '''
    dataset = []
    for condition in os.listdir(path):
        if condition not in ['fractured', 'not fractured']:
            continue

        label = 1 if condition == 'fractured' else 0
        path_condition = os.path.join(path, condition)

        for img in os.listdir(path_condition):
            img_path = os.path.join(path_condition, img)
            if os.path.isfile(img_path):
                dataset.append({
                    'label': label,
                    'img_path': img_path
                })

    return dataset

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# =============================
# Set dataset paths
# =============================
train_path = '/content/drive/MyDrive/BoneFractureDataset/training'
test_path = '/content/drive/MyDrive/BoneFractureDataset/testing'

In [8]:
# List subdirectories (assumed to be class names)
classes = os.listdir(train_path)

# Count files in each class folder
class_distribution = {}
for cls in classes:
    cls_path = os.path.join(train_path, cls)
    if os.path.isdir(cls_path):
        count = len([f for f in os.listdir(cls_path) if os.path.isfile(os.path.join(cls_path, f))])
        class_distribution[cls] = count

# Print results
for cls, count in class_distribution.items():
    print(f"{cls}: {count} images")

dataset_train = load_path(train_path)
df_train = pd.DataFrame(dataset_train)
df_train.head()

dataset_test = load_path('/content/drive/MyDrive/BoneFractureDataset/testing')
df_test = pd.DataFrame(dataset_test)
df_test.head()

fractured: 4490 images
not_fractured: 4383 images


,label,img_path
0,1,/content/drive/MyDrive/BoneFractureDataset/tes...
1,1,/content/drive/MyDrive/BoneFractureDataset/tes...
2,1,/content/drive/MyDrive/BoneFractureDataset/tes...
3,1,/content/drive/MyDrive/BoneFractureDataset/tes...
4,1,/content/drive/MyDrive/BoneFractureDataset/tes...


In [9]:
path = '/content/drive/MyDrive/BoneFractureDataset'

df_train.to_csv('dataset_train', index=False)
train_image_paths_csv = "train_image_paths.csv"

In [10]:
import shutil
import os

src = 'train_image_paths_csv.csv'
dst = '/content/drive/MyDrive/BoneFractureDataset/train_image_paths.csv'


In [11]:
# Make sure the source file exists
if os.path.exists(src):
    shutil.copy(src, dst)  # Use copy instead of move
    os.remove(src)         # Then remove the source file manually
    print("File successfully copied and original deleted.")
else:
    print(f"Source file '{src}' not found.")

train_images_paths = pd.read_csv(os.path.join(path,train_image_paths_csv),dtype=str,header=None)
train_images_paths.columns = ['label','image_path']

train_images_paths.head() # Rename the selected column

train_images_paths['label'] = train_images_paths['image_path'].map(lambda x:'positive' if 'positive' in x else 'negative')
train_images_paths.head()

Source file 'train_image_paths_csv.csv' not found.


,label,image_path
0,negative,img_path
1,negative,/content/drive/MyDrive/BoneFractureDataset/tra...
2,negative,/content/drive/MyDrive/BoneFractureDataset/tra...
3,negative,/content/drive/MyDrive/BoneFractureDataset/tra...
4,negative,/content/drive/MyDrive/BoneFractureDataset/tra...


In [12]:
"""Test set"""

path = '/content/drive/MyDrive/BoneFractureDataset'
df_test.to_csv('test_image_paths_csv.csv', index=False)
test_image_paths_csv = "test_image_paths.csv"
test_data_paths = pd.read_csv(os.path.join(path,test_image_paths_csv), dtype=str, header=None)
test_data_paths.columns = ['label', 'image_path']
test_data_paths['label'] = test_data_paths['image_path'].map(lambda x:'positive' if 'positive' in x else 'negative')
test_data_paths.head()

train_images_paths['label_index']= train_images_paths.label

train_images_paths.label_index.replace('positive', 1, inplace=True)
train_images_paths.label_index.replace('negative', 0, inplace=True)
train_images_paths.head(3)

test_data_paths['label_index']= test_data_paths.label
test_data_paths.label_index.replace('positive', 1, inplace=True)
test_data_paths.label_index.replace('negative', 0, inplace=True)
test_data_paths.head(3)

/tmp/ipython-input-1466538735.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_images_paths.label_index.replace('positive', 1, inplace=True)
/tmp/ipython-input-1466538735.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, i

,label,image_path,label_index
0,negative,img_path,0
1,negative,/content/drive/MyDrive/BoneFractureDataset/tes...,0
2,negative,/content/drive/MyDrive/BoneFractureDataset/tes...,0


In [13]:
"""## Image Preprocessing for creation of Dataset."""

im_size = 256

"""#### Random Rotation flip"""

def random_rotation_flip(image,size = 256):
    if random.randint(0,1):
        image = cv2.flip(image,1) # 1-->horizontal flip 0-->Vertical flip -1-->Horizontal and vertical

    if random.randint(0,1):
            angle = random.randint(-30,30)
            M = cv2.getRotationMatrix2D((size/2,size/2),angle,1)
            #The third parameter: the size of the transformed image
            image = cv2.warpAffine(image,M,(size,size))
    return image

"""#### Image / Dataset Loader with random rotation flip"""

def image_loader(Path, size = 224):
    Images = []
    for path in tqdm(Path):
        try:
            image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image,(size,size))
            image = random_rotation_flip(image,size)
            Images.append(image)
        except Exception as e:
            print(str(e))
    Images = np.asarray(Images).astype('float32')
    mean = np.mean(Images)
    std = np.std(Images)
    Images = (Images - mean) / std

    return Images

In [14]:
import os
os.chdir('/content/drive/My Drive/BoneFractureDataset')

X_train = image_loader(train_images_paths['image_path'][:50,],im_size)

y_train = train_images_paths['label']

Y_train = y_train.replace("positive",1)
Y_train = Y_train.replace("negative",0)

X_test = image_loader(test_data_paths['image_path'][:50,],im_size)

y_test = test_data_paths['label']

Y_test = y_test.replace("positive",1)
Y_test = Y_test.replace("negative",0)

train_images_paths.head()

train, test = train_test_split(train_images_paths, test_size=0.2,random_state=seed)

 26%|██▌       | 13/50 [00:00<00:00, 67.98it/s]

OpenCV(4.12.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



100%|██████████| 50/50 [00:00<00:00, 95.58it/s] 
/tmp/ipython-input-1183495970.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y_train = Y_train.replace("negative",0)
  0%|          | 0/50 [00:00<?, ?it/s]

OpenCV(4.12.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



100%|██████████| 50/50 [00:19<00:00,  2.57it/s]
/tmp/ipython-input-1183495970.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y_test = Y_test.replace("negative",0)


In [15]:
"""**Dataset Preparation**"""

image_generator_settings = dict(
                          rescale = 1. / 255,
                          #samplewise_center = True,
                          #samplewise_std_normalization = True
                          #rotation_range = 5,
                         )
image_generator = ImageDataGenerator(**image_generator_settings)

path='/content/drive/My Drive/BoneFractureDataset/Bones1'

print(train.columns)
print(test.columns)

# Convert labels to string explicitly for binary classification
train['label'] = train['label'].map({0: 'not fractured', 1: 'fractured'}).astype(str)
test['label'] = test['label'].map({0: 'not fractured', 1: 'fractured'}).astype(str)

train_generator = image_generator.flow_from_dataframe(dataframe = train,directory = None,x_col = 'image_path',y_col = 'label_index',batch_size = 64,shuffle = True,class_mode = 'raw', target_size = (im_size, im_size),color_mode = 'rgb',interpolation='nearest',validate_filenames=False,seed=seed)
test_generator = image_generator.flow_from_dataframe(dataframe = test,directory = None,x_col = 'image_path',y_col = 'label_index',batch_size = 64,shuffle = False,class_mode = 'raw', target_size = (im_size, im_size),color_mode = 'rgb',interpolation='nearest', validate_filenames=True,seed=seed)

CLASSES = 2
input_shape = (im_size,im_size,3)

Index(['label', 'image_path', 'label_index'], dtype='object')
Index(['label', 'image_path', 'label_index'], dtype='object')
Found 3592 non-validated image filenames.
Found 899 validated image filenames.


In [16]:
"""**Helper function for Multiple Mertrics for Evaluation for our model**"""

def metrics():
    return [
        AUC(name="auc"),
        BinaryAccuracy("accuracy"),
        Precision(name="precision"),
        Recall(name="recall")
    ]

#new
def plot_history(hs, epochs, metric):
    if metric == 'loss':
        label_y = 'Loss'
    elif metric == 'accuracy':
        label_y = 'Accuracy'
    elif metric == 'auc':
        label_y = 'AUC'
    else:
        label_y = 'Kappa'

    plt.rcParams['figure.figsize'] = [15, 8]
    plt.rcParams['font.size'] = 16
    plt.clf()
    for label in hs:
        epoch_count = len(hs[label].history[metric])
        plt.plot(np.arange(1, epoch_count + 1), hs[label].history[metric], label='{0:s} train {1:s}'.format(label, metric), linewidth=2)
        plt.plot(np.arange(1, epoch_count + 1), hs[label].history['val_{0:s}'.format(metric)], label='{0:s} validation {1:s}'.format(label, metric), linewidth=2)
    plt.ylim((0, 1))
    plt.xlabel('Epochs')
    plt.ylabel(label_y)
    plt.legend()
    plt.show()

def print_results(model_name, epochs, hs, model_eval):
    print("Train Loss     : {0:.5f}".format(hs.history['loss'][-1]))
    print("Validation Loss: {0:.5f}".format(hs.history['val_loss'][-1]))
    print("Test Loss      : {0:.5f}".format(model_eval[0]))
    print("---")
    print("Train AUC     : {0:.5f}".format(hs.history['auc'][-1]))
    print("Validation AUC: {0:.5f}".format(hs.history['val_auc'][-1]))
    print("Test AUC      : {0:.5f}".format(model_eval[1]))
    print("---")
    print("Train Accuracy     : {0:.5f}".format(hs.history['accuracy'][-1]))
    print("Validation Accuracy: {0:.5f}".format(hs.history['val_accuracy'][-1]))
    print("Test Accuracy      : {0:.5f}".format(model_eval[2]))
    print("---")
    print("Train Kappa     : {0:.5f}".format(hs.history['kappa'][-1]))
    print("Validation Kappa: {0:.5f}".format(hs.history['kappa'][-1]))
    print("Test Kappa      : {0:.5f}".format(model_eval[3]))
    # Plot train and validation error per epoch.
    plot_history(hs={model_name: hs}, epochs=epochs, metric='loss')
    plot_history(hs={model_name: hs}, epochs=epochs, metric='auc')
    plot_history(hs={model_name: hs}, epochs=epochs, metric='accuracy')
    plot_history(hs={model_name: hs}, epochs=epochs, metric='kappa')

In [17]:
""" **Helper function to clear Memory Utilization enabling it to Work with GPU and standard RAM allowance in Kaggle.**"""

def clean_up(model):
    K.clear_session()
    del model
    gc.collect() #garbage collection
    print("...Finished clean up...")

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameters
im_size = 256
batch_size = 64
seed = 42

# Define data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,               # Normalize pixel values
    horizontal_flip=True,         # Data augmentation
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.8,1.2],
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1./255                # Only normalize for test data
)


In [18]:
# Load training data
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/BoneFractureDataset/training',           # Path to your training data
    target_size=(im_size, im_size),
    batch_size=batch_size,
    class_mode='binary',          # Binary classification
    seed=seed
)

# Load testing data
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/BoneFractureDataset/testing',            # Path to your testing data
    target_size=(im_size, im_size),
    batch_size=batch_size,
    class_mode='binary',          # Binary classification
    seed=seed
)

Found 8873 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [19]:
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC # Import necessary metrics

# ========== METRICS ==========
def metrics():
    return [
        BinaryAccuracy(name="accuracy"),
        Precision(name="precision"),
        Recall(name="recall"),
        AUC(name="auc")
    ]


# ========== MODEL BUILDER ==========
def resnet_builder(pooling="avg", shape=(256, 256, 3), trainable_layers_after=None):
    resNet = ResNet152V2(
        include_top=False,
        weights='imagenet',
        input_shape=shape,
        pooling=pooling
    )
    if trainable_layers_after:
        for layer in resNet.layers[:trainable_layers_after]:
            layer.trainable = False
    else:
        resNet.trainable = False

    model = Sequential([
        resNet,
        Dense(512, activation='relu', name='dense_layer_1'),
        Dropout(0.5, name='dropout_1'),
        Dense(256, activation='relu', name='dense_layer_2'),
        Dropout(0.2, name='dropout_2'),
        Dense(1, activation="sigmoid", name="resnet_output_sigmoid")
    ], name="resnet_with_dense")

    return model

In [21]:

# ========== COMPILE & TRAIN ==========
epochs = 25

early_stop = EarlyStopping(
    monitor="val_auc",  # More stable metric than val_loss
    mode="max",
    patience=7,
    restore_best_weights=True
)

resnet_model = resnet_builder(pooling='avg')

resnet_model.compile(
    optimizer=Adam(),
    loss=BinaryCrossentropy(from_logits=False),
    metrics=metrics()
)

# ========== MODEL TRAINING ==========
hs = resnet_model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=epochs,
    callbacks=[early_stop]
)

print('\n✅ Finished training.')
print('------------------')
resnet_model.summary()

Epoch 1/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 7064s 51s/step - accuracy: 0.6492 - auc: 0.7051 - loss: 0.6741 - precision: 0.6341 - recall: 0.6641 - val_accuracy: 0.7317 - val_auc: 0.8373 - val_loss: 0.5906 - val_precision: 0.5995 - val_recall: 0.9917
Epoch 2/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 6695s 48s/step - accuracy: 0.8307 - auc: 0.9163 - loss: 0.3640 - precision: 0.8148 - recall: 0.8543 - val_accuracy: 0.7017 - val_auc: 0.7377 - val_loss: 0.7769 - val_precision: 0.5818 - val_recall: 0.9042
Epoch 3/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 6556s 47s/step - accuracy: 0.8860 - auc: 0.9528 - loss: 0.2782 - precision: 0.8741 - recall: 0.8981 - val_accuracy: 0.6900 - val_auc: 0.7556 - val_loss: 0.6288 - val_precision: 0.5978 - val_recall: 0.6875
Epoch 4/25
139/139 ━━━━━━━━━━━━━━━━━━━━ 6622s 48s/step - accuracy: 0.9004 - auc: 0.9655 - loss: 0.2372 - precision: 0.8888 - recall: 0.9145 - val_accuracy: 0.7117 - val_auc: 0.8029 - val_loss: 0.6190 - val_precision: 0.6414 - val_recall: 0.6333
Epoch 5/25
139/139 ━

KeyboardInterrupt: 

In [ ]:
# ========== SAVE MODEL ==========
filename = '/kaggle/working/model_5.h5'
resnet_model.save(filename)
print('💾 Model saved to:', filename)


In [ ]:
# ========== EVALUATE MODEL ==========
resnet_model_eval = resnet_model.evaluate(test_generator)
print_results(model_name='ResNet', epochs=epochs, hs=hs, model_eval=resnet_model_eval)

In [ ]:
# ========== OPTIONAL DEBUG: View predictions ==========
x_batch, y_batch = next(iter(test_generator))
y_pred_batch = resnet_model.predict(x_batch)

print("\n🔍 Sample Predictions vs. True Labels")
print("Predictions:", np.round(y_pred_batch[:10].flatten(), 2))
print("True labels:", y_batch[:10])

In [ ]:
def plot_history(hs, epochs, metric):
    if metric == 'loss':
        label_y = 'Loss'
    elif metric == 'accuracy':
        label_y = 'Accuracy'
    elif metric == 'auc':
        label_y = 'AUC'
    else:
        label_y = 'Kappa'

    plt.rcParams['figure.figsize'] = [15, 8]
    plt.rcParams['font.size'] = 16
    plt.clf()
    for label in hs:
        epoch_count = len(hs[label].history[metric])
        plt.plot(
            np.arange(1, epoch_count + 1),
            hs[label].history[metric],
            label=f'{label} train {metric}',
            linewidth=2
        )
        plt.plot(
            np.arange(1, epoch_count + 1),
            hs[label].history[f'val_{metric}'],
            label=f'{label} validation {metric}',
            linewidth=2
        )
    plt.ylim((0, 1))
    plt.xlabel('Epochs')
    plt.ylabel(label_y)
    plt.legend()
    plt.show()


In [ ]:
#new
def plot_history(hs, epochs, metric):
    if metric == 'loss':
        label_y = 'Loss'
    elif metric == 'accuracy':
        label_y = 'Accuracy'
    elif metric == 'auc':
        label_y = 'AUC'
    else:
        label_y = 'Kappa'

    plt.rcParams['figure.figsize'] = [15, 8]
    plt.rcParams['font.size'] = 16
    plt.clf()
    for label in hs:
        epoch_count = len(hs[label].history[metric])
        plt.plot(np.arange(1, epoch_count + 1), hs[label].history[metric], label='{0:s} train {1:s}'.format(label, metric), linewidth=2)
        plt.plot(np.arange(1, epoch_count + 1), hs[label].history['val_{0:s}'.format(metric)], label='{0:s} validation {1:s}'.format(label, metric), linewidth=2)
    plt.ylim((0, 1))
    plt.xlabel('Epochs')
    plt.ylabel(label_y)
    plt.legend()
    plt.show()

def print_results(model_name, epochs, hs, model_eval):
    print("Train Loss     : {0:.5f}".format(hs.history['loss'][-1]))
    print("Validation Loss: {0:.5f}".format(hs.history['val_loss'][-1]))
    print("Test Loss      : {0:.5f}".format(model_eval[0]))
    print("---")
    print("Train AUC     : {0:.5f}".format(hs.history['auc'][-1]))
    print("Validation AUC: {0:.5f}".format(hs.history['val_auc'][-1]))
    print("Test AUC      : {0:.5f}".format(model_eval[1]))
    print("---")
    print("Train Accuracy     : {0:.5f}".format(hs.history['accuracy'][-1]))
    print("Validation Accuracy: {0:.5f}".format(hs.history['val_accuracy'][-1]))
    print("Test Accuracy      : {0:.5f}".format(model_eval[2]))
    print("---")
    print("Train Kappa     : {0:.5f}".format(hs.history['kappa'][-1]))
    print("Validation Kappa: {0:.5f}".format(hs.history['kappa'][-1]))
    print("Test Kappa      : {0:.5f}".format(model_eval[3]))
    # Plot train and validation error per epoch.
    plot_history(hs={model_name: hs}, epochs=epochs, metric='loss')
    plot_history(hs={model_name: hs}, epochs=epochs, metric='auc')
    plot_history(hs={model_name: hs}, epochs=epochs, metric='accuracy')
    plot_history(hs={model_name: hs}, epochs=epochs, metric='kappa')